## Comparison to BLAST

#### Install Blast tools

* https://blast.ncbi.nlm.nih.gov/doc/blast-help/downloadblastdata.html
* https://www.ncbi.nlm.nih.gov/books/NBK279690/

#### Compare our prediction with BLAST Tools

In [11]:
import pandas as pd
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import json, os
from tqdm import tqdm
import sklearn.metrics as metrics
from BLAST.utils import metric

In [19]:
df = pd.read_csv('./BLAST/src/testset.csv')

In [20]:
df.head()

,id,sequence,length,class
0,Thhalv10001938m -- EIL,MMMFNEMGMYGKMDFFSSTSLGEIDVCPLPQAEQDHPVVEEDYTDD...,582,EIL
1,Tp1g37880 -- EIL,MGVFFSDPDSIAEDDGYTDNELDVDELEKRIWKQEMRLRRLKEHRK...,211,EIL
2,Pbr004574.1 -- EIL,MGDVGEIGPDISSDIEEDLRCDNIAEKDVSDEEIEAEDLERRMWKD...,608,EIL
3,Sme2.5_02278.1_g00005.1 -- EIL,MNNEVVEENQEFDDEEINYDDLKRRMWKDRMRMQILKGKKRDMMIE...,494,EIL
4,ONIVA11G15950.1 -- EIL,MDASKKSVMTKEEQQLSPAASPAAAVMTAEADAINEEQDKAAAATT...,466,EIL


In [21]:
df.shape

(64091, 4)

In [4]:
records = []
for i, row in df.iterrows():
    record = SeqRecord(Seq(row['sequence']), id=str(i+1), description=" | " + row['class'])
    records.append(record)

# Save to a FASTA file
output_file = "./BLAST/testset.fasta"
SeqIO.write(records, output_file, "fasta")

print(f"FASTA file saved as {output_file}")

FASTA file saved as ./BLAST/testset.fasta


#### Build reference(train) database for blast

In [5]:
ref_pah = "../data/mix_data/trainset"

In [6]:
gene_info_path = "../data/gene_info.json"
with open(gene_info_path, 'r') as json_file:
    gene_info = json.load(json_file)

In [7]:
records = []
for gene, info in tqdm(gene_info.items()):
    path = f"{ref_pah}/{info['file_code']}.csv"
    df_gene = pd.read_csv(path)
    df_gene = df_gene[df_gene['class'] == 1]
    
    # write
    for _, row in df_gene.iterrows():
        record = SeqRecord(Seq(row['sequence']), id=row['id'] + "--" + gene, description="")
        records.append(record)

# Save to a FASTA file
output_file = "./BLAST/trainset.fasta"
SeqIO.write(records, output_file, "fasta")
print(f"FASTA file saved as {output_file}")

100%|██████████| 58/58 [00:08<00:00,  6.97it/s]


FASTA file saved as ./BLAST/trainset.fasta


In [8]:
import sys, os
current_directory = os.getcwd()
root_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
sys.path.append(root_directory)

from processing.fasta import read as fasta_read
x=fasta_read("./BLAST/trainset.fasta", family=1)
x

,id,sequence,length,class
0,GSMUA_AchrUn_randomP23040_001--AP2,MNMNASDNWLDFSLSQQRCLLEAFSDAPHHHGLSLSLSLSTNAATE...,469,1
1,KHN36319.1--AP2,MNIGVPSHLHQTQPSSAAVEAVPPNFYHHTPLHNYGLYYELEGEHV...,548,1
2,678453786--AP2,MGSSQEHQSWLDFSLTNNNHLSFSSQSTCAAAAPVGGYHHRFSNLE...,315,1
3,OBART06G22270.1--AP2,MAATFYGVGSIALAMHEDDEEEGSGRVFGFAAGDLVRPAVVTQQLF...,424,1
4,MA_30905g0010--AP2,MGCHVQILVAGENNNLKAGEESSASLKSEAAAKNGSSLSQPKTKRA...,420,1
...,...,...,...,...
256274,XP_010522314.1--bZIP,MEPNDRPSHCFNLMEGNPSQNNHFNSLFLPTNTESHVQLHSVSPNT...,187,1
256275,Sp_092640_iogy.t2--bZIP,MMEDNNQIKCNNNNSSNRNRVPLMDDGELEFSNQELLSSANIGEIP...,290,1
256276,Vradi06g16770.1--bZIP,MEGVDQSNMKRNGSVFGSSSGANLRPPNRLNTAVPVSQLGVAENQK...,553,1
256277,Gorai.009G212600.2--bZIP,MGTNMNFGSNPPPSGDGGGNKPPGNNLLTRQPSIYSLTFDEFQSTM...,424,1


In [9]:
x.shape

(256279, 4)

#### BUILD DATABASE FOR BLAST

In [10]:
!makeblastdb -in ./BLAST/trainset.fasta -dbtype prot -out ./BLAST/database/pygenomics_ref_db



Building a new DB, current time: 11/06/2025 02:34:09
New DB name:   /Users/genereux/Documents/PHD/MegaPlantTF/test/BLAST/database/pygenomics_ref_db
New DB title:  ./BLAST/trainset.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /Users/genereux/Documents/PHD/MegaPlantTF/test/BLAST/database/pygenomics_ref_db
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 256279 sequences in 6.56908 seconds.




#### RUN BLAST SEARCH TOOL

In [12]:
!blastp -query ./BLAST/testset.fasta -db ./BLAST/database/pygenomics_ref_db -out ./BLAST/blast_result.txt -outfmt 6 -evalue 1e-5 -num_threads 10

#### View Blast Analysis results

In [2]:
columns = [
    "query_id", "subject_id", "perc_identity", "alignment_length",
    "mismatches", "gap_opens", "q_start", "q_end", "s_start", "s_end",
    "evalue", "bit_score"
]
blast_results = pd.read_csv("./BLAST/blast_result.txt", sep="\t", names=columns)
#blast_results.to_csv("./BLAST/blast_results.csv", index=False)

# Display the first few rows of the DataFrame
blast_results

,query_id,subject_id,perc_identity,alignment_length,mismatches,gap_opens,q_start,q_end,s_start,s_end,evalue,bit_score
0,1,676714038--EIL,89.696,592,49,6,1,581,1,591,0.000000e+00,1063.0
1,1,XP_013629559.1--EIL,84.628,592,59,10,1,581,1,571,0.000000e+00,1001.0
2,1,GSBRNA2T00066199001--EIL,83.277,592,59,10,1,581,1,563,0.000000e+00,980.0
3,1,XP_009133914.1--EIL,85.135,592,56,10,1,581,1,571,0.000000e+00,977.0
4,1,XP_009133916.1--EIL,85.135,592,56,10,1,581,1,571,0.000000e+00,977.0
...,...,...,...,...,...,...,...,...,...,...,...,...
33292098,64091,XP_015882438.1--NF-YA,40.670,209,105,6,65,260,3,205,3.220000e-35,127.0
33292099,64091,Pbr037486.1--NF-YA,43.787,169,79,2,112,270,45,207,3.300000e-35,127.0
33292100,64091,Pp3c7_7470V3.2.p--NF-YA,46.795,156,75,4,121,269,94,248,3.410000e-35,130.0
33292101,64091,Pp3c7_7470V3.3.p--NF-YA,46.795,156,75,4,121,269,94,248,3.410000e-35,130.0


In [3]:
blast_results

,query_id,subject_id,perc_identity,alignment_length,mismatches,gap_opens,q_start,q_end,s_start,s_end,evalue,bit_score
0,1,676714038--EIL,89.696,592,49,6,1,581,1,591,0.000000e+00,1063.0
1,1,XP_013629559.1--EIL,84.628,592,59,10,1,581,1,571,0.000000e+00,1001.0
2,1,GSBRNA2T00066199001--EIL,83.277,592,59,10,1,581,1,563,0.000000e+00,980.0
3,1,XP_009133914.1--EIL,85.135,592,56,10,1,581,1,571,0.000000e+00,977.0
4,1,XP_009133916.1--EIL,85.135,592,56,10,1,581,1,571,0.000000e+00,977.0
...,...,...,...,...,...,...,...,...,...,...,...,...
33292098,64091,XP_015882438.1--NF-YA,40.670,209,105,6,65,260,3,205,3.220000e-35,127.0
33292099,64091,Pbr037486.1--NF-YA,43.787,169,79,2,112,270,45,207,3.300000e-35,127.0
33292100,64091,Pp3c7_7470V3.2.p--NF-YA,46.795,156,75,4,121,269,94,248,3.410000e-35,130.0
33292101,64091,Pp3c7_7470V3.3.p--NF-YA,46.795,156,75,4,121,269,94,248,3.410000e-35,130.0


#### Proccessing to make final inference base on threshold

In [7]:
voting_threshold = 80
unique_query_ids = sorted(blast_results['query_id'].unique())

predictions = ["Unknown"  for _ in range(len(df))]
for query_id in tqdm(unique_query_ids, desc="Max Voting : "):
    filtered_df = blast_results[blast_results['query_id'] == query_id]
    
    # Find the row with the highest perc_identity greater than voting_threshold
    best_match = filtered_df[filtered_df['perc_identity'] > voting_threshold].sort_values(by='perc_identity', ascending=False).head(1)
    if not best_match.empty:
        subject_id = best_match.iloc[0]['subject_id']
        subject_id_part = subject_id.split('--')[-1]
        predictions[query_id-1] = (subject_id_part)

blast_prediction = pd.DataFrame({'prediction': predictions})

Max Voting : 100%|██████████| 64037/64037 [18:50<00:00, 56.67it/s]


In [8]:
with open("./BLAST/src/class_mapping.json", 'r') as json_file:
    class_mapping = json.load(json_file)
true_label = pd.read_csv("./BLAST/src/true_labels.csv")

In [9]:
blast_prediction["predicted_label"] = blast_prediction.prediction.map(class_mapping)
blast_prediction.head()

,prediction,predicted_label
0,EIL,1
1,EIL,1
2,EIL,1
3,EIL,1
4,EIL,1


In [17]:
blast_prediction.to_csv("./results/BLAST/blast_predictions.csv", index=False)

#### F1 SCORE

In [12]:
import pandas as pd
import json
from sklearn.metrics import f1_score, accuracy_score

id_to_class = {v: k for k, v in class_mapping.items()}

y_true = true_label["true_label"]
y_pred = blast_prediction["predicted_label"]
families = sorted(set(y_true) & set(y_pred))  # only common labels
metrics_dict = {}

for fam in families:
    y_true_bin = (y_true == fam).astype(int)
    y_pred_bin = (y_pred == fam).astype(int)
    f1 = f1_score(y_true_bin, y_pred_bin, zero_division=0)
    acc = accuracy_score(y_true_bin, y_pred_bin)
    metrics_dict[id_to_class[fam]] = {
        "f1": float(f1),
        "accuracy": float(acc)
    }

output_path = "./results/BLAST/blast_family_metrics_id80.json"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with open(output_path, "w") as f:
    json.dump(metrics_dict, f, indent=4)

print(f"✅ Saved BLAST per-family metrics to {output_path}")


✅ Saved BLAST per-family metrics to ./results/BLAST/blast_family_metrics_id80.json


#### Display performance

In [18]:
metric.show_metrics(true_label, blast_prediction.predicted_label) #80% identity percentage (70_90 when aligning especially short domaine specific proteins)

In [ ]:
metric.show_metrics(true_label, blast_prediction.predicted_label) #50% identity percentage

In [14]:
import importlib
importlib.reload(metric)

<module 'BLAST.utils.metric' from '/Users/genereux/Documents/PHD/MegaPlantTF/test/BLAST/utils/metric.py'>

In [ ]:
metric.show_confusion(true_label, blast_prediction.predicted_label, class_mapping)